In [1]:
%load_ext autoreload
%autoreload 2
from utils import *

In [2]:
df = pd.read_csv("csv/cars.csv")

In [3]:
df.sample(5)

,buying,maint,doors,persons,lug_boot,safety,class
1197,med,low,2,4,small,low,unacc
1324,low,vhigh,3,2,small,med,unacc
187,vhigh,high,4,more,big,med,unacc
1312,low,vhigh,2,4,big,med,acc
415,vhigh,low,5more,4,small,med,unacc


In [4]:
df.columns

Index(['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class'], dtype='object')

In [5]:
print("buying : ", np.unique(df['buying'].values))
print("maint : ", np.unique(df['maint'].values))
print("doors : ", np.unique(df['doors'].values))
print("persons : ", np.unique(df['persons'].values))
print("lug_boot : ", np.unique(df['lug_boot'].values))
print("safety : ", np.unique(df['safety'].values))
print("class : ", np.unique(df['class'].values))

buying :  ['high' 'low' 'med' 'vhigh']
maint :  ['high' 'low' 'med' 'vhigh']
doors :  ['2' '3' '4' '5more']
persons :  ['2' '4' 'more']
lug_boot :  ['big' 'med' 'small']
safety :  ['high' 'low' 'med']
class :  ['acc' 'good' 'unacc' 'vgood']


In [6]:
df.loc[df['class'] == 'unacc','class'] = 1
df.loc[df['class'] == 'acc','class'] = 2
df.loc[df['class'] == 'good','class'] = 3
df.loc[df['class'] == 'vgood','class'] = 4

In [7]:
df.loc[df['buying'] == 'low','buying'] = 1
df.loc[df['buying'] == 'med','buying'] = 2
df.loc[df['buying'] == 'high','buying'] = 3
df.loc[df['buying'] == 'vhigh','buying'] = 4

In [8]:
df.loc[df['maint'] == 'low','maint'] = 1
df.loc[df['maint'] == 'med','maint'] = 2
df.loc[df['maint'] == 'high','maint'] = 3
df.loc[df['maint'] == 'vhigh','maint'] = 4

In [9]:
df.loc[df['doors'] == '2','doors'] = 1
df.loc[df['doors'] == '3','doors'] = 2
df.loc[df['doors'] == '4','doors'] = 3
df.loc[df['doors'] == '5more','doors'] = 4

In [10]:
df.loc[df['persons'] == '2','persons'] = 1
df.loc[df['persons'] == '4','persons'] = 2
df.loc[df['persons'] == 'more','persons'] = 3

In [11]:
df.loc[df['lug_boot'] == 'small','lug_boot'] = 1
df.loc[df['lug_boot'] == 'med','lug_boot'] = 2
df.loc[df['lug_boot'] == 'big','lug_boot'] = 3

In [12]:
df.loc[df['safety'] == 'low','safety'] = 1
df.loc[df['safety'] == 'med','safety'] = 2
df.loc[df['safety'] == 'high','safety'] = 3

In [13]:
print("buying : ", np.unique(df['buying'].values))
print("maint : ", np.unique(df['maint'].values))
print("doors : ", np.unique(df['doors'].values))
print("persons : ", np.unique(df['persons'].values))
print("lug_boot : ", np.unique(df['lug_boot'].values))
print("safety : ", np.unique(df['safety'].values))
print("class : ", np.unique(df['class'].values))

buying :  [1 2 3 4]
maint :  [1 2 3 4]
doors :  [1 2 3 4]
persons :  [1 2 3]
lug_boot :  [1 2 3]
safety :  [1 2 3]
class :  [1 2 3 4]


In [14]:
g = Log()
h = Sum()

# rank Shannon discrimination measure
f_r = Dsr()
rsdm = Gdm(h, g, f_r)

# conditional Shannon entropy
f = Ds()
sdm = Gdm(h, g, f)

# pessimistic rank discrimination measure
f_p = Mindsr()
g_p = Frac() 
prdm = Gdm(h, g_p, f_p)

# rank Gini discrimination measure 
g_g = One_minus()
rgdm = Gdm(h, g_g, f_r)

# H_M
h_m = Max()
g_m = One_minus_square()
H_m = Gdm(h_m, g_m, f_r)

# Gini discrimination measure
gdm = Gdm(h, g_g, f)

# H_Q
f_q = Avgdsr()
h_q = Square_root()
H_q = Gdm(h_q, g_g, f_q)

In [15]:
vectors = df.values

In [16]:
dataset = LabeledSet(6)
dataset.addExamples(vectors[:,0:6], vectors[:,6:])

In [17]:
NMI1(dataset)

0.01403699950674473

In [18]:
folds = get_folds(dataset, 4)

In [19]:
nb_folds = 4

In [20]:
acc_list = []
std_acc_list = []
depth_list = []
std_depth_list = []
leaves_list =  []
std_leaves_list = []
ratio_list = []
std_ratio_list = []
pairs_list = []
std_pairs_list = []
evaluations_list = []
std_eval_list = []
pratio_list = []
std_pratio_list = []
nb_examples_list = []
std_nb_examples_list = []

In [21]:
# Rank Shannon discrimination measure
acc = []
leaves = []
depth = []
ratio = []
pairs = []
evaluation = []
p_ratio = []
nb_examples = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(6)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(rsdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3, 4])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
    evaluation.append(tree.evaluate_monotonicity())
    p_ratio.append(tree.pairs_ratio())
    nb_examples.append(tree.get_total_examples_ratio())    
    
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)
avg_eval = np.mean(evaluation)
std_eval = np.std(evaluation)
avg_pratio = np.mean(p_ratio)
std_pratio = np.std(p_ratio)
avg_nb_examples = np.mean(nb_examples)
std_nb_examples = np.std(nb_examples)

acc_list.append(avg_accuracy)
std_acc_list.append(std_accuracy)

depth_list.append(avg_depth)
std_depth_list.append(std_depth)

leaves_list.append(avg_leaves)
std_leaves_list.append(std_leaves)

ratio_list.append(avg_ratio)
std_ratio_list.append(std_ratio)

pairs_list.append(avg_pairs)
std_pairs_list.append(std_pairs)

evaluations_list.append(avg_eval)
std_eval_list.append(std_eval)

pratio_list.append(avg_pratio)
std_pratio_list.append(std_pratio)

nb_examples_list.append(avg_nb_examples)
std_nb_examples_list.append(std_nb_examples)
# print("average accuracy : ", avg_accuracy)
# print("standard deviation of accuracy : ", std_accuracy)
# print("average depth : ", avg_depth)
# print("standard deviation of depth : ", std_depth)
# print("average number of leaves : ", avg_leaves)
# print("standard deviation of number of leaves : ", std_leaves)
# print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
# print("standard deviation of ratio : ", std_ratio)
# print("average number of pairs used for ratio computing : ", avg_pairs)
# print("standard deviation of number of pairs : ", std_pairs)

In [22]:
# Shannon entropy
acc = []
leaves = []
depth = []
ratio = []
pairs = []
evaluation = []
p_ratio = []
nb_examples = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(6)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(sdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3, 4])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
    evaluation.append(tree.evaluate_monotonicity())
    p_ratio.append(tree.pairs_ratio())
    nb_examples.append(tree.get_total_examples_ratio())    
    
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)
avg_eval = np.mean(evaluation)
std_eval = np.std(evaluation)
avg_pratio = np.mean(p_ratio)
std_pratio = np.std(p_ratio)
avg_nb_examples = np.mean(nb_examples)
std_nb_examples = np.std(nb_examples)

acc_list.append(avg_accuracy)
std_acc_list.append(std_accuracy)

depth_list.append(avg_depth)
std_depth_list.append(std_depth)

leaves_list.append(avg_leaves)
std_leaves_list.append(std_leaves)

ratio_list.append(avg_ratio)
std_ratio_list.append(std_ratio)

pairs_list.append(avg_pairs)
std_pairs_list.append(std_pairs)

evaluations_list.append(avg_eval)
std_eval_list.append(std_eval)

pratio_list.append(avg_pratio)
std_pratio_list.append(std_pratio)

nb_examples_list.append(avg_nb_examples)
std_nb_examples_list.append(std_nb_examples)

In [23]:
# Rank Gini discrimination measure
acc = []
leaves = []
depth = []
ratio = []
pairs = []
evaluation = []
p_ratio = []
nb_examples = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(6)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(rgdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3, 4])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
    evaluation.append(tree.evaluate_monotonicity())
    p_ratio.append(tree.pairs_ratio())
    nb_examples.append(tree.get_total_examples_ratio())    
    
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)
avg_eval = np.mean(evaluation)
std_eval = np.std(evaluation)
avg_pratio = np.mean(p_ratio)
std_pratio = np.std(p_ratio)
avg_nb_examples = np.mean(nb_examples)
std_nb_examples = np.std(nb_examples)

acc_list.append(avg_accuracy)
std_acc_list.append(std_accuracy)

depth_list.append(avg_depth)
std_depth_list.append(std_depth)

leaves_list.append(avg_leaves)
std_leaves_list.append(std_leaves)

ratio_list.append(avg_ratio)
std_ratio_list.append(std_ratio)

pairs_list.append(avg_pairs)
std_pairs_list.append(std_pairs)

evaluations_list.append(avg_eval)
std_eval_list.append(std_eval)

pratio_list.append(avg_pratio)
std_pratio_list.append(std_pratio)

nb_examples_list.append(avg_nb_examples)
std_nb_examples_list.append(std_nb_examples)
# print("average accuracy : ", avg_accuracy)
# print("standard deviation of accuracy : ", std_accuracy)
# print("average depth : ", avg_depth)
# print("standard deviation of depth : ", std_depth)
# print("average number of leaves : ", avg_leaves)
# print("standard deviation of number of leaves : ", std_leaves)
# print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
# print("standard deviation of ratio : ", std_ratio)
# print("average number of pairs used for ratio computing : ", avg_pairs)
# print("standard deviation of number of pairs : ", std_pairs)

In [24]:
# Gini discrimination measure
acc = []
leaves = []
depth = []
ratio = []
pairs = []
evaluation = []
p_ratio = []
nb_examples = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(6)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(gdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3, 4])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
    evaluation.append(tree.evaluate_monotonicity())
    p_ratio.append(tree.pairs_ratio())
    nb_examples.append(tree.get_total_examples_ratio())    
    
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)
avg_eval = np.mean(evaluation)
std_eval = np.std(evaluation)
avg_pratio = np.mean(p_ratio)
std_pratio = np.std(p_ratio)
avg_nb_examples = np.mean(nb_examples)
std_nb_examples = np.std(nb_examples)

acc_list.append(avg_accuracy)
std_acc_list.append(std_accuracy)

depth_list.append(avg_depth)
std_depth_list.append(std_depth)

leaves_list.append(avg_leaves)
std_leaves_list.append(std_leaves)

ratio_list.append(avg_ratio)
std_ratio_list.append(std_ratio)

pairs_list.append(avg_pairs)
std_pairs_list.append(std_pairs)

evaluations_list.append(avg_eval)
std_eval_list.append(std_eval)

pratio_list.append(avg_pratio)
std_pratio_list.append(std_pratio)

nb_examples_list.append(avg_nb_examples)
std_nb_examples_list.append(std_nb_examples)
# print("average accuracy : ", avg_accuracy)
# print("standard deviation of accuracy : ", std_accuracy)
# print("average depth : ", avg_depth)
# print("standard deviation of depth : ", std_depth)
# print("average number of leaves : ", avg_leaves)
# print("standard deviation of number of leaves : ", std_leaves)
# print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
# print("standard deviation of ratio : ", std_ratio)
# print("average number of pairs used for ratio computing : ", avg_pairs)
# print("standard deviation of number of pairs : ", std_pairs)

In [25]:
# Pessimistic rank discrimination measure
acc = []
leaves = []
depth = []
ratio = []
pairs = []
evaluation = []
p_ratio = []
nb_examples = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(6)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(prdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3, 4])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
    evaluation.append(tree.evaluate_monotonicity())
    p_ratio.append(tree.pairs_ratio())
    nb_examples.append(tree.get_total_examples_ratio())    
    
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)
avg_eval = np.mean(evaluation)
std_eval = np.std(evaluation)
avg_pratio = np.mean(p_ratio)
std_pratio = np.std(p_ratio)
avg_nb_examples = np.mean(nb_examples)
std_nb_examples = np.std(nb_examples)

acc_list.append(avg_accuracy)
std_acc_list.append(std_accuracy)

depth_list.append(avg_depth)
std_depth_list.append(std_depth)

leaves_list.append(avg_leaves)
std_leaves_list.append(std_leaves)

ratio_list.append(avg_ratio)
std_ratio_list.append(std_ratio)

pairs_list.append(avg_pairs)
std_pairs_list.append(std_pairs)

evaluations_list.append(avg_eval)
std_eval_list.append(std_eval)

pratio_list.append(avg_pratio)
std_pratio_list.append(std_pratio)

nb_examples_list.append(avg_nb_examples)
std_nb_examples_list.append(std_nb_examples)
# print("average accuracy : ", avg_accuracy)
# print("standard deviation of accuracy : ", std_accuracy)
# print("average depth : ", avg_depth)
# print("standard deviation of depth : ", std_depth)
# print("average number of leaves : ", avg_leaves)
# print("standard deviation of number of leaves : ", std_leaves)
# print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
# print("standard deviation of ratio : ", std_ratio)
# print("average number of pairs used for ratio computing : ", avg_pairs)
# print("standard deviation of number of pairs : ", std_pairs)

In [26]:
# H_M
acc = []
leaves = []
depth = []
ratio = []
pairs = []
evaluation = []
p_ratio = []
nb_examples = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(6)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(H_m, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3, 4])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
    evaluation.append(tree.evaluate_monotonicity())
    p_ratio.append(tree.pairs_ratio())
    nb_examples.append(tree.get_total_examples_ratio())    
    
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)
avg_eval = np.mean(evaluation)
std_eval = np.std(evaluation)
avg_pratio = np.mean(p_ratio)
std_pratio = np.std(p_ratio)
avg_nb_examples = np.mean(nb_examples)
std_nb_examples = np.std(nb_examples)

acc_list.append(avg_accuracy)
std_acc_list.append(std_accuracy)

depth_list.append(avg_depth)
std_depth_list.append(std_depth)

leaves_list.append(avg_leaves)
std_leaves_list.append(std_leaves)

ratio_list.append(avg_ratio)
std_ratio_list.append(std_ratio)

pairs_list.append(avg_pairs)
std_pairs_list.append(std_pairs)

evaluations_list.append(avg_eval)
std_eval_list.append(std_eval)

pratio_list.append(avg_pratio)
std_pratio_list.append(std_pratio)

nb_examples_list.append(avg_nb_examples)
std_nb_examples_list.append(std_nb_examples)
# print("average accuracy : ", avg_accuracy)
# print("standard deviation of accuracy : ", std_accuracy)
# print("average depth : ", avg_depth)
# print("standard deviation of depth : ", std_depth)
# print("average number of leaves : ", avg_leaves)
# print("standard deviation of number of leaves : ", std_leaves)
# print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
# print("standard deviation of ratio : ", std_ratio)
# print("average number of pairs used for ratio computing : ", avg_pairs)
# print("standard deviation of number of pairs : ", std_pairs)

In [27]:
# H_Q
acc = []
leaves = []
depth = []
ratio = []
pairs = []
evaluation = []
p_ratio = []
nb_examples = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(6)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(H_q, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3, 4])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
    evaluation.append(tree.evaluate_monotonicity())
    p_ratio.append(tree.pairs_ratio())
    nb_examples.append(tree.get_total_examples_ratio())    
    
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)
avg_eval = np.mean(evaluation)
std_eval = np.std(evaluation)
avg_pratio = np.mean(p_ratio)
std_pratio = np.std(p_ratio)
avg_nb_examples = np.mean(nb_examples)
std_nb_examples = np.std(nb_examples)

acc_list.append(avg_accuracy)
std_acc_list.append(std_accuracy)

depth_list.append(avg_depth)
std_depth_list.append(std_depth)

leaves_list.append(avg_leaves)
std_leaves_list.append(std_leaves)

ratio_list.append(avg_ratio)
std_ratio_list.append(std_ratio)

pairs_list.append(avg_pairs)
std_pairs_list.append(std_pairs)

evaluations_list.append(avg_eval)
std_eval_list.append(std_eval)

pratio_list.append(avg_pratio)
std_pratio_list.append(std_pratio)

nb_examples_list.append(avg_nb_examples)
std_nb_examples_list.append(std_nb_examples)
# print("average accuracy : ", avg_accuracy)
# print("standard deviation of accuracy : ", std_accuracy)
# print("average depth : ", avg_depth)
# print("standard deviation of depth : ", std_depth)
# print("average number of leaves : ", avg_leaves)
# print("standard deviation of number of leaves : ", std_leaves)
# print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
# print("standard deviation of ratio : ", std_ratio)
# print("average number of pairs used for ratio computing : ", avg_pairs)
# print("standard deviation of number of pairs : ", std_pairs)

In [28]:
s_cci = ""
for i in range(7):
    s_cci += "& " + "{0:.2f}".format(acc_list[i] * 100) + " \% $\pm$ " + "{0:.2f}".format(std_acc_list[i]) + " \% "

s_depth = ""
for i in range(7):
    s_depth += "& " + "{0:.2f}".format(depth_list[i]) + " $\pm$ " + "{0:.2f}".format(std_depth_list[i]) + ""

s_leaves = ""
for i in range(7):
    s_leaves += "& " + "{0:.2f}".format(leaves_list[i]) + " $\pm$ " + "{0:.2f}".format(std_leaves_list[i]) + ""

s_ratio = ""
for i in range(7):
    s_ratio += "& " + "{0:.2f}".format(ratio_list[i] * 100) + " \% $\pm$ " + "{0:.2f}".format(std_ratio_list[i] * 100) + " \%"
    
s_pairs = ""
for i in range(7):
    s_pairs += "& " + "{0:.2f}".format(pairs_list[i]) + " $\pm$ " + "{0:.2f}".format(std_pairs_list[i]) + ""

s_eval = ""
for i in range(7):
    s_eval += "& " + "{0:.2f}".format(evaluations_list[i]) + " $\pm$ " + "{0:.2f}".format(std_eval_list[i]) + ""

s_pratio = ""
for i in range(7):
    s_pratio += "& " + "{0:.2f}".format(pratio_list[i]) + " $\pm$ " + "{0:.2f}".format(std_pratio_list[i]) + ""

s_nb_examples = ""
for i in range(7):
    s_nb_examples += "& " + "{0:.2f}".format(nb_examples_list[i]) + " $\pm$ " + "{0:.2f}".format(std_nb_examples_list[i]) + ""

In [29]:
print("CCI ", s_cci)
print("avgDepth ", s_depth)
print("avgLeaves ", s_leaves)
print("avgRatio ", s_ratio)
print("avgNbPairs ", s_pairs)
print("avgEval ", s_eval)
print("avgPairRatio ", s_pratio)
print("avgNbExamples ", s_nb_examples)

CCI  & 81.95 \% $\pm$ 0.05 \% & 81.44 \% $\pm$ 0.06 \% & 81.60 \% $\pm$ 0.05 \% & 78.25 \% $\pm$ 0.09 \% & 84.43 \% $\pm$ 0.02 \% & 83.56 \% $\pm$ 0.03 \% & 63.94 \% $\pm$ 0.05 \% 
avgDepth  & 10.50 $\pm$ 0.50& 10.75 $\pm$ 0.43& 10.50 $\pm$ 0.50& 10.25 $\pm$ 0.43& 10.00 $\pm$ 0.00& 11.00 $\pm$ 0.00& 11.00 $\pm$ 0.00
avgLeaves  & 66.50 $\pm$ 13.16& 37.25 $\pm$ 1.30& 66.50 $\pm$ 13.16& 38.00 $\pm$ 1.58& 52.00 $\pm$ 3.94& 54.00 $\pm$ 3.74& 121.25 $\pm$ 14.04
avgRatio  & 82.17 \% $\pm$ 5.00 \%& 83.82 \% $\pm$ 1.39 \%& 82.76 \% $\pm$ 5.56 \%& 84.68 \% $\pm$ 1.84 \%& 79.25 \% $\pm$ 2.93 \%& 77.33 \% $\pm$ 2.90 \%& 52.19 \% $\pm$ 4.34 \%
avgNbPairs  & 27.75 $\pm$ 7.36& 11.75 $\pm$ 1.09& 27.75 $\pm$ 7.36& 12.00 $\pm$ 1.41& 18.25 $\pm$ 1.92& 18.00 $\pm$ 2.12& 46.75 $\pm$ 8.79
avgEval  & 0.03 $\pm$ 0.00& 0.01 $\pm$ 0.00& 0.03 $\pm$ 0.00& 0.01 $\pm$ 0.01& 0.02 $\pm$ 0.00& 0.01 $\pm$ 0.00& 0.01 $\pm$ 0.00
avgPairRatio  & 0.42 $\pm$ 0.03& 0.32 $\pm$ 0.02& 0.42 $\pm$ 0.03& 0.32 $\pm$ 0.03& 0.36 $\pm

In [30]:
s_nb_examples

'& 462.00 $\\pm$ 57.47& 261.00 $\\pm$ 44.65& 460.75 $\\pm$ 57.85& 259.00 $\\pm$ 65.86& 355.75 $\\pm$ 33.24& 355.25 $\\pm$ 31.59& 806.50 $\\pm$ 86.62'

In [31]:
s_cci

'& 81.95 \\% $\\pm$ 0.05 \\% & 81.44 \\% $\\pm$ 0.06 \\% & 81.60 \\% $\\pm$ 0.05 \\% & 78.25 \\% $\\pm$ 0.09 \\% & 84.43 \\% $\\pm$ 0.02 \\% & 83.56 \\% $\\pm$ 0.03 \\% & 63.94 \\% $\\pm$ 0.05 \\% '

In [32]:
s_depth

'& 10.50 $\\pm$ 0.50& 10.75 $\\pm$ 0.43& 10.50 $\\pm$ 0.50& 10.25 $\\pm$ 0.43& 10.00 $\\pm$ 0.00& 11.00 $\\pm$ 0.00& 11.00 $\\pm$ 0.00'

In [33]:
s_leaves

'& 66.50 $\\pm$ 13.16& 37.25 $\\pm$ 1.30& 66.50 $\\pm$ 13.16& 38.00 $\\pm$ 1.58& 52.00 $\\pm$ 3.94& 54.00 $\\pm$ 3.74& 121.25 $\\pm$ 14.04'

In [34]:
s_ratio

'& 82.17 \\% $\\pm$ 5.00 \\%& 83.82 \\% $\\pm$ 1.39 \\%& 82.76 \\% $\\pm$ 5.56 \\%& 84.68 \\% $\\pm$ 1.84 \\%& 79.25 \\% $\\pm$ 2.93 \\%& 77.33 \\% $\\pm$ 2.90 \\%& 52.19 \\% $\\pm$ 4.34 \\%'

In [35]:
s_pairs

'& 27.75 $\\pm$ 7.36& 11.75 $\\pm$ 1.09& 27.75 $\\pm$ 7.36& 12.00 $\\pm$ 1.41& 18.25 $\\pm$ 1.92& 18.00 $\\pm$ 2.12& 46.75 $\\pm$ 8.79'

In [36]:
s_eval

'& 0.03 $\\pm$ 0.00& 0.01 $\\pm$ 0.00& 0.03 $\\pm$ 0.00& 0.01 $\\pm$ 0.01& 0.02 $\\pm$ 0.00& 0.01 $\\pm$ 0.00& 0.01 $\\pm$ 0.00'

In [37]:
s_pratio

'& 0.42 $\\pm$ 0.03& 0.32 $\\pm$ 0.02& 0.42 $\\pm$ 0.03& 0.32 $\\pm$ 0.03& 0.36 $\\pm$ 0.02& 0.34 $\\pm$ 0.03& 0.39 $\\pm$ 0.03'

In [38]:
s_nb_examples

'& 462.00 $\\pm$ 57.47& 261.00 $\\pm$ 44.65& 460.75 $\\pm$ 57.85& 259.00 $\\pm$ 65.86& 355.75 $\\pm$ 33.24& 355.25 $\\pm$ 31.59& 806.50 $\\pm$ 86.62'